In [ ]:
import numpy as np
import arviz as az
import math
# import pymc3 as pm
import pickle
import matplotlib.pyplot as plt
from google.protobuf import text_format
from scipy.stats import multivariate_normal as mvn
from scipy.stats import skewnorm
from scipy.stats import norm
from scipy.interpolate import griddata
import pp_mix.protos.py.params_pb2 as params_pb2

from pp_mix.interface import cluster_estimate
from pp_mix.utils import loadChains, to_numpy, to_proto
from pp_mix.protos.py.state_pb2 import MultivariateMixtureState, EigenVector, EigenMatrix
from pp_mix.protos.py.params_pb2 import Params

In [ ]:
chain = loadChains("data/data_by_rule_student/chains_compare_CD/chain.recordio", MultivariateMixtureState)
acc_rates = np.loadtxt("data/data_by_rule_student/chains_compare_CD/acc_rates.csv", delimiter=',')


check chain mixing on some variables


In [ ]:
tau_chain = np.array([x.lamb_block.tau for x in chain])
plt.plot(tau_chain)



In [ ]:
first_sbar_chain = np.array([to_numpy(x.sigma_bar)[0] for x in chain])
plt.plot(first_sbar_chain,color='red')

last_sbar_chain = np.array([to_numpy(x.sigma_bar)[-1] for x in chain])
plt.plot(last_sbar_chain,color='blue')


In [ ]:
first_sbar_chain = np.array([to_numpy(x.sigma_bar)[0] for x in chain])
plt.plot(first_sbar_chain)

last_sbar_chain = np.array([to_numpy(x.sigma_bar)[-1] for x in chain])
plt.plot(last_sbar_chain)


evaluate performance on simulate data: return performance dataframe


In [ ]:
from scipy import stats
from sklearn.metrics import adjusted_rand_score
import pandas as pd
from math import sqrt

list_performance = list()

n_cluster_chain = np.array([x.ma for x in chain])
n_nonall_chain = np.array([x.mna for x in chain])

post_mode_nclus = stats.mode(n_cluster_chain)[0][0] # store in dataframe
post_avg_nclus = n_cluster_chain.mean() # store in dataframe
post_avg_nonall =  n_nonall_chain.mean() # store in dataframe

clus_alloc_chain = [x.clus_alloc for x in chain]
best_clus = cluster_estimate(np.array(clus_alloc_chain))
n_clus_best_clus = np.size(np.unique(best_clus))

#true_clus = np.repeat(range(M),n_percluster)
#ari_best_clus = adjusted_rand_score(true_clus, best_clus) # store in dataframe

#aris_chain = np.array([adjusted_rand_score(true_clus, x) for x in clus_alloc_chain])
#mean_aris, sigma_aris = np.mean(aris_chain), np.std(aris_chain) # store mean_aris in dataframe
#CI_aris = stats.norm.interval(0.95, loc=mean_aris, scale=sigma_aris/sqrt(len(aris_chain))) # store in dataframe

list_performance.append([acc_rates[0],acc_rates[1],post_mode_nclus,
                    post_avg_nclus, post_avg_nonall])
        
        
df_performance = pd.DataFrame(list_performance, columns=('means_ar','lambda_ar',
                                      'mode_nclus', 'avg_nclus', 'avg_nonalloc'))
    

In [ ]:
# read the dataset
df = pd.read_excel("41598_2021_2025_MOESM2_ESM.xlsx")
# extract only right eyes (OD)
df_od = df.loc[df['Masked_Id_Age1'].str.contains("OD", case=True)].set_index('Masked_Id_Age1')
# clustering obtained by cifu (method used in the paper)
cifu_clustering = df_od['Cluster_Id_Age1']

# THIS IS THE DATA USED IN THE ALGORITHM, THEN CONVERTED TO NUMPY ARRAY
df_od = df_od[df_od.columns[1:]]
# NUMPY ARRAY OF DATA
data = df_od.to_numpy()

In [ ]:
vals = df_od[df_od.columns[2:]].values


In [ ]:
theta = np.linspace(0, 2 * np.pi, vals.shape[1])
dx = theta[1] - theta[0]

In [9]:
fig, axs = plt.subplots(nrows=sqrt(n_clus_best_clus), ncols=sqrt(n_clus_best_clus), polar=True)


for clus, ax in zip(range(n_clus_best_clus), axs.ravel()):
  ax.plot(theta, vals[best_clus == clus,:].T)
  # chart formatting
  ax.set_title("cluster ", clus)
  ax.get_legend().remove()

plt.show()

0
1
2
